# Monitoreo y Detección de Data Drift - Pipeline Genérico

Este notebook implementa un sistema completo de monitoreo de modelos y detección de drift en datos.

**Funcionalidades:**
- Muestreo periódico de datos para análisis estadístico
- Cálculo de métricas de data drift (KS test, PSI, Jensen-Shannon, Chi-cuadrado)
- Comparación de distribuciones históricas vs actuales
- Generación de alertas automáticas cuando se detectan desviaciones significativas
- Análisis temporal de la evolución del drift
- Sistema de recomendaciones para retraining

## 1. Importar Librerías Necesarias

Importar todas las librerías requeridas para el monitoreo y detección de drift.

In [1]:
import warnings
import pandas as pd
import numpy as np
import json
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import pickle

warnings.filterwarnings('ignore')

# Estadísticas y tests
from scipy import stats
from scipy.spatial.distance import jensenshannon
from scipy.stats import ks_2samp, chi2_contingency

# Configurar estilo de gráficos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Importaciones completadas exitosamente")

✅ Importaciones completadas exitosamente


## 2. Cargar Datos Históricos y Actuales

Cargar dataset de referencia (histórico) y dataset actual para comparación.

In [2]:
# Cargar configuración
config_path = "../../config.json"
data_path = None

if os.path.exists(config_path):
    with open(config_path, 'r') as f:
        config = json.load(f)
        data_path = config.get('data_path', 'Base_de_datos.csv')
else:
    data_path = "../../alzheimers_disease_data.csv"

# Cargar dataset completo
df_full = pd.read_csv(data_path)

print("="*80)
print("CARGA DE DATOS PARA MONITOREO")
print("="*80)
print(f"\n✓ Dataset completo cargado desde: {data_path}")
print(f"  Total de registros: {len(df_full)}")

# Simular datos históricos (primeros 80%) y datos actuales (últimos 20%)
# En producción, estos serían datos de diferentes períodos de tiempo
split_point = int(len(df_full) * 0.8)

df_reference = df_full.iloc[:split_point].copy()  # Datos históricos (referencia)
df_current = df_full.iloc[split_point:].copy()     # Datos actuales (monitoreo)

print(f"\n📊 División de datos:")
print(f"   Datos de referencia (históricos): {len(df_reference)} registros ({len(df_reference)/len(df_full)*100:.1f}%)")
print(f"   Datos actuales (monitoreo): {len(df_current)} registros ({len(df_current)/len(df_full)*100:.1f}%)")

# Excluir columnas de identificación
exclude_cols = ['PatientID', 'DoctorInCharge']
feature_cols = [col for col in df_full.columns if col not in exclude_cols]

print(f"\n✓ Columnas a monitorear: {len(feature_cols)}")
print(f"   Columnas excluidas: {exclude_cols}")

CARGA DE DATOS PARA MONITOREO

✓ Dataset completo cargado desde: ../../alzheimers_disease_data.csv
  Total de registros: 2149

📊 División de datos:
   Datos de referencia (históricos): 1719 registros (80.0%)
   Datos actuales (monitoreo): 430 registros (20.0%)

✓ Columnas a monitorear: 33
   Columnas excluidas: ['PatientID', 'DoctorInCharge']


## 3. Funciones de Cálculo de Data Drift

Implementar funciones reutilizables para calcular métricas de drift.

In [3]:
def calculate_psi(expected, actual, bins=10):
    """
    Calcular Population Stability Index (PSI) para variables numéricas.
    
    PSI < 0.1: No hay cambio significativo
    0.1 <= PSI < 0.2: Cambio moderado
    PSI >= 0.2: Cambio significativo (requiere atención)
    
    Parámetros:
    -----------
    expected : array-like
        Distribución de referencia (histórica)
    actual : array-like
        Distribución actual
    bins : int
        Número de bins para discretización
    
    Retorna:
    --------
    float : Valor de PSI
    """
    # Eliminar NaN
    expected = expected[~np.isnan(expected)]
    actual = actual[~np.isnan(actual)]
    
    # Crear bins basados en la distribución esperada
    breakpoints = np.quantile(expected, np.linspace(0, 1, bins + 1))
    breakpoints = np.unique(breakpoints)  # Eliminar duplicados
    
    if len(breakpoints) < 2:
        return 0.0
    
    # Calcular frecuencias
    expected_counts = np.histogram(expected, bins=breakpoints)[0]
    actual_counts = np.histogram(actual, bins=breakpoints)[0]
    
    # Evitar divisiones por cero
    expected_percents = (expected_counts + 1) / (len(expected) + len(breakpoints) - 1)
    actual_percents = (actual_counts + 1) / (len(actual) + len(breakpoints) - 1)
    
    # Calcular PSI
    psi = np.sum((actual_percents - expected_percents) * np.log(actual_percents / expected_percents))
    
    return psi


def calculate_ks_statistic(reference, current):
    """
    Calcular Kolmogorov-Smirnov test para variables numéricas.
    
    Parámetros:
    -----------
    reference : array-like
        Datos de referencia
    current : array-like
        Datos actuales
    
    Retorna:
    --------
    tuple : (statistic, p-value)
    """
    reference = reference[~np.isnan(reference)]
    current = current[~np.isnan(current)]
    
    return ks_2samp(reference, current)


def calculate_jensen_shannon(reference, current, bins=30):
    """
    Calcular Jensen-Shannon divergence para variables numéricas.
    
    JS = 0: Distribuciones idénticas
    JS = 1: Distribuciones completamente diferentes
    
    Parámetros:
    -----------
    reference : array-like
        Datos de referencia
    current : array-like
        Datos actuales
    bins : int
        Número de bins para discretización
    
    Retorna:
    --------
    float : Divergencia de Jensen-Shannon
    """
    reference = reference[~np.isnan(reference)]
    current = current[~np.isnan(current)]
    
    # Crear bins comunes
    min_val = min(reference.min(), current.min())
    max_val = max(reference.max(), current.max())
    bins_edges = np.linspace(min_val, max_val, bins + 1)
    
    # Calcular histogramas normalizados
    ref_hist, _ = np.histogram(reference, bins=bins_edges)
    cur_hist, _ = np.histogram(current, bins=bins_edges)
    
    # Normalizar (evitar ceros)
    ref_hist = (ref_hist + 1e-10) / (ref_hist.sum() + 1e-10 * len(ref_hist))
    cur_hist = (cur_hist + 1e-10) / (cur_hist.sum() + 1e-10 * len(cur_hist))
    
    # Calcular JS divergence
    js_div = jensenshannon(ref_hist, cur_hist)
    
    return js_div


def calculate_chi_square(reference, current):
    """
    Calcular test Chi-cuadrado para variables categóricas.
    
    Parámetros:
    -----------
    reference : array-like
        Datos categóricos de referencia
    current : array-like
        Datos categóricos actuales
    
    Retorna:
    --------
    tuple : (chi2_statistic, p-value, cramers_v)
    """
    # Obtener todas las categorías únicas
    all_categories = list(set(reference) | set(current))
    
    # Crear tabla de contingencia
    ref_counts = pd.Series(reference).value_counts()
    cur_counts = pd.Series(current).value_counts()
    
    contingency_table = pd.DataFrame({
        'reference': [ref_counts.get(cat, 0) for cat in all_categories],
        'current': [cur_counts.get(cat, 0) for cat in all_categories]
    })
    
    # Chi-cuadrado test
    chi2, p_value, dof, expected = chi2_contingency(contingency_table.T)
    
    # Cramér's V (medida de efecto)
    n = contingency_table.sum().sum()
    cramers_v = np.sqrt(chi2 / (n * (min(contingency_table.shape) - 1)))
    
    return chi2, p_value, cramers_v


print("✅ Funciones de drift implementadas:")
print("   - calculate_psi() - Population Stability Index")
print("   - calculate_ks_statistic() - Kolmogorov-Smirnov test")
print("   - calculate_jensen_shannon() - Jensen-Shannon divergence")
print("   - calculate_chi_square() - Chi-cuadrado test")

✅ Funciones de drift implementadas:
   - calculate_psi() - Population Stability Index
   - calculate_ks_statistic() - Kolmogorov-Smirnov test
   - calculate_jensen_shannon() - Jensen-Shannon divergence
   - calculate_chi_square() - Chi-cuadrado test


## 4. Calcular Métricas de Drift por Variable

Analizar cada variable y calcular métricas de drift apropiadas.

In [4]:
print("\n" + "="*80)
print("ANÁLISIS DE DATA DRIFT POR VARIABLE")
print("="*80 + "\n")

# Clasificar variables
numeric_cols = df_reference[feature_cols].select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = df_reference[feature_cols].select_dtypes(include=['object']).columns.tolist()

print(f"📊 Variables numéricas a monitorear: {len(numeric_cols)}")
print(f"📝 Variables categóricas a monitorear: {len(categorical_cols)}\n")

# Resultados de drift
drift_results = []

# Analizar variables numéricas
print("🔍 Analizando variables numéricas...")
for col in numeric_cols:
    ref_data = df_reference[col].dropna().values
    cur_data = df_current[col].dropna().values
    
    if len(ref_data) > 0 and len(cur_data) > 0:
        # Calcular métricas
        psi = calculate_psi(ref_data, cur_data)
        ks_stat, ks_pval = calculate_ks_statistic(ref_data, cur_data)
        js_div = calculate_jensen_shannon(ref_data, cur_data)
        
        # Determinar nivel de alerta basado en PSI
        if psi < 0.1:
            alert = "✅ OK"
        elif psi < 0.2:
            alert = "⚠️ MODERADO"
        else:
            alert = "🚨 CRÍTICO"
        
        drift_results.append({
            'Variable': col,
            'Tipo': 'Numérica',
            'PSI': round(psi, 4),
            'KS_Statistic': round(ks_stat, 4),
            'KS_PValue': round(ks_pval, 4),
            'JS_Divergence': round(js_div, 4),
            'Chi2': None,
            'Chi2_PValue': None,
            'Cramers_V': None,
            'Alerta': alert
        })

print(f"✓ {len(numeric_cols)} variables numéricas analizadas\n")

# Analizar variables categóricas
print("🔍 Analizando variables categóricas...")
for col in categorical_cols:
    ref_data = df_reference[col].dropna().values
    cur_data = df_current[col].dropna().values
    
    if len(ref_data) > 0 and len(cur_data) > 0:
        # Calcular test Chi-cuadrado
        chi2, chi2_pval, cramers_v = calculate_chi_square(ref_data, cur_data)
        
        # Determinar nivel de alerta basado en Cramér's V
        if cramers_v < 0.1:
            alert = "✅ OK"
        elif cramers_v < 0.3:
            alert = "⚠️ MODERADO"
        else:
            alert = "🚨 CRÍTICO"
        
        drift_results.append({
            'Variable': col,
            'Tipo': 'Categórica',
            'PSI': None,
            'KS_Statistic': None,
            'KS_PValue': None,
            'JS_Divergence': None,
            'Chi2': round(chi2, 4),
            'Chi2_PValue': round(chi2_pval, 4),
            'Cramers_V': round(cramers_v, 4),
            'Alerta': alert
        })

print(f"✓ {len(categorical_cols)} variables categóricas analizadas\n")

# Crear DataFrame de resultados
drift_df = pd.DataFrame(drift_results)

print("="*80)
print("RESUMEN DE DRIFT - TODAS LAS VARIABLES")
print("="*80)
print(drift_df.to_string(index=False))
print("="*80)


ANÁLISIS DE DATA DRIFT POR VARIABLE

📊 Variables numéricas a monitorear: 33
📝 Variables categóricas a monitorear: 0

🔍 Analizando variables numéricas...
✓ 33 variables numéricas analizadas

🔍 Analizando variables categóricas...
✓ 0 variables categóricas analizadas

RESUMEN DE DRIFT - TODAS LAS VARIABLES
                 Variable     Tipo    PSI  KS_Statistic  KS_PValue  JS_Divergence Chi2 Chi2_PValue Cramers_V Alerta
                      Age Numérica 0.0181        0.0299     0.9069         0.1101 None        None      None   ✅ OK
                   Gender Numérica 0.0000        0.0253     0.9755         0.0179 None        None      None   ✅ OK
                Ethnicity Numérica 0.0011        0.0104     1.0000         0.0120 None        None      None   ✅ OK
           EducationLevel Numérica 0.0234        0.0701     0.0638         0.0586 None        None      None   ✅ OK
                      BMI Numérica 0.0691        0.0777     0.0294         0.1180 None        None      None   ✅ O

## 5. Identificar Variables con Drift Crítico

Filtrar y mostrar variables que requieren atención inmediata.

In [5]:
print("\n" + "="*80)
print("🚨 ALERTAS DE DRIFT CRÍTICO")
print("="*80 + "\n")

# Filtrar variables con alerta crítica o moderada
critical_vars = drift_df[drift_df['Alerta'] == "🚨 CRÍTICO"]
moderate_vars = drift_df[drift_df['Alerta'] == "⚠️ MODERADO"]
ok_vars = drift_df[drift_df['Alerta'] == "✅ OK"]

print(f"📊 Resumen de alertas:")
print(f"   🚨 Variables críticas: {len(critical_vars)}")
print(f"   ⚠️  Variables moderadas: {len(moderate_vars)}")
print(f"   ✅ Variables OK: {len(ok_vars)}\n")

if len(critical_vars) > 0:
    print("🚨 VARIABLES CON DRIFT CRÍTICO (requieren atención inmediata):")
    print("="*80)
    for _, row in critical_vars.iterrows():
        print(f"\n   Variable: {row['Variable']} ({row['Tipo']})")
        if row['Tipo'] == 'Numérica':
            print(f"      PSI: {row['PSI']} (umbral crítico: >= 0.2)")
            print(f"      KS Statistic: {row['KS_Statistic']}")
            print(f"      JS Divergence: {row['JS_Divergence']}")
        else:
            print(f"      Cramér's V: {row['Cramers_V']} (umbral crítico: >= 0.3)")
            print(f"      Chi2 p-value: {row['Chi2_PValue']}")
    print("\n" + "="*80)
else:
    print("✅ No se detectaron variables con drift crítico.\n")

if len(moderate_vars) > 0:
    print("\n⚠️  VARIABLES CON DRIFT MODERADO (monitorear):")
    print("="*80)
    for _, row in moderate_vars.iterrows():
        print(f"   • {row['Variable']} ({row['Tipo']})")
    print("="*80)

# Guardar resultados
drift_df.to_csv("../../drift_report.csv", index=False)
print(f"\n💾 Reporte de drift guardado en: drift_report.csv")


🚨 ALERTAS DE DRIFT CRÍTICO

📊 Resumen de alertas:
   🚨 Variables críticas: 0
   ⚠️  Variables moderadas: 0
   ✅ Variables OK: 33

✅ No se detectaron variables con drift crítico.


💾 Reporte de drift guardado en: drift_report.csv


## 6. Visualización de Distribuciones

Comparar distribuciones históricas vs actuales para variables con drift.

In [ ]:
print("\n" + "="*80)
print("VISUALIZACIÓN DE DISTRIBUCIONES")
print("="*80 + "\n")

# Visualizar las variables con drift crítico o moderado (máximo 6)
vars_to_plot = pd.concat([critical_vars, moderate_vars]).head(6)

if len(vars_to_plot) > 0:
    n_vars = len(vars_to_plot)
    n_cols = 2
    n_rows = (n_vars + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(14, 5 * n_rows))
    axes = axes.flatten() if n_vars > 1 else [axes]
    
    for idx, (_, row) in enumerate(vars_to_plot.iterrows()):
        col_name = row['Variable']
        ax = axes[idx]
        
        if row['Tipo'] == 'Numérica':
            # Histogramas superpuestos para variables numéricas
            ref_data = df_reference[col_name].dropna()
            cur_data = df_current[col_name].dropna()
            
            ax.hist(ref_data, bins=30, alpha=0.6, label='Referencia', color='blue', density=True)
            ax.hist(cur_data, bins=30, alpha=0.6, label='Actual', color='red', density=True)
            
            ax.set_xlabel(col_name, fontweight='bold')
            ax.set_ylabel('Densidad', fontweight='bold')
            ax.set_title(f'{col_name} - {row["Alerta"]}\nPSI: {row["PSI"]:.4f}', 
                        fontsize=11, fontweight='bold')
            ax.legend()
            ax.grid(alpha=0.3)
            
        else:
            # Gráfico de barras para variables categóricas
            ref_data = df_reference[col_name].value_counts(normalize=True)
            cur_data = df_current[col_name].value_counts(normalize=True)
            
            # Combinar categorías
            all_cats = list(set(ref_data.index) | set(cur_data.index))
            x = np.arange(len(all_cats))
            width = 0.35
            
            ref_vals = [ref_data.get(cat, 0) for cat in all_cats]
            cur_vals = [cur_data.get(cat, 0) for cat in all_cats]
            
            ax.bar(x - width/2, ref_vals, width, label='Referencia', alpha=0.8, color='blue')
            ax.bar(x + width/2, cur_vals, width, label='Actual', alpha=0.8, color='red')
            
            ax.set_xlabel('Categorías', fontweight='bold')
            ax.set_ylabel('Frecuencia Relativa', fontweight='bold')
            ax.set_title(f'{col_name} - {row["Alerta"]}\nCramér\'s V: {row["Cramers_V"]:.4f}', 
                        fontsize=11, fontweight='bold')
            ax.set_xticks(x)
            ax.set_xticklabels(all_cats, rotation=45, ha='right')
            ax.legend()
            ax.grid(axis='y', alpha=0.3)
    
    # Ocultar ejes no utilizados
    for idx in range(n_vars, len(axes)):
        axes[idx].set_visible(False)
    
    plt.tight_layout()
    plt.show()
    
    print(f"✅ Visualizadas {n_vars} variables con drift")
else:
    print("✅ No hay variables con drift significativo para visualizar")

## 7. Panel de Métricas de Drift

Dashboard visual con indicadores de alerta tipo semáforo.

In [ ]:
print("\n" + "="*80)
print("PANEL DE INDICADORES - SEMÁFORO DE DRIFT")
print("="*80 + "\n")

# Crear figura con múltiples subplots
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# 1. Gráfico de dona - Estado general
ax1 = fig.add_subplot(gs[0, 0])
sizes = [len(ok_vars), len(moderate_vars), len(critical_vars)]
labels = ['OK', 'Moderado', 'Crítico']
colors = ['#2ecc71', '#f39c12', '#e74c3c']
explode = (0.05, 0.05, 0.1)

wedges, texts, autotexts = ax1.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%',
                                     explode=explode, startangle=90, textprops={'fontweight': 'bold'})
ax1.set_title('Estado General del Sistema\n(% Variables por Nivel)', fontsize=12, fontweight='bold')

# 2. Barras horizontales - Top variables con mayor drift
ax2 = fig.add_subplot(gs[0, 1])
top_drift = drift_df[drift_df['Tipo'] == 'Numérica'].nlargest(10, 'PSI')
if len(top_drift) > 0:
    colors_bars = ['#e74c3c' if x >= 0.2 else '#f39c12' if x >= 0.1 else '#2ecc71' 
                   for x in top_drift['PSI']]
    ax2.barh(top_drift['Variable'], top_drift['PSI'], color=colors_bars, alpha=0.8)
    ax2.axvline(x=0.1, color='orange', linestyle='--', linewidth=2, label='Umbral Moderado')
    ax2.axvline(x=0.2, color='red', linestyle='--', linewidth=2, label='Umbral Crítico')
    ax2.set_xlabel('PSI (Population Stability Index)', fontweight='bold')
    ax2.set_title('Top 10 Variables - PSI', fontsize=12, fontweight='bold')
    ax2.legend()
    ax2.grid(axis='x', alpha=0.3)

# 3. Heatmap de métricas numéricas
ax3 = fig.add_subplot(gs[1, :])
numeric_drift = drift_df[drift_df['Tipo'] == 'Numérica'].copy()
if len(numeric_drift) > 0:
    # Seleccionar top 15 variables por PSI
    numeric_drift_top = numeric_drift.nlargest(15, 'PSI')
    
    # Crear matriz de métricas
    metrics_matrix = numeric_drift_top[['PSI', 'KS_Statistic', 'JS_Divergence']].T
    
    im = ax3.imshow(metrics_matrix, cmap='RdYlGn_r', aspect='auto', vmin=0, vmax=0.5)
    
    # Configurar ejes
    ax3.set_xticks(np.arange(len(numeric_drift_top)))
    ax3.set_xticklabels(numeric_drift_top['Variable'], rotation=45, ha='right', fontsize=9)
    ax3.set_yticks(np.arange(3))
    ax3.set_yticklabels(['PSI', 'KS Stat', 'JS Div'], fontweight='bold')
    ax3.set_title('Heatmap de Métricas de Drift (Variables Numéricas)', fontsize=12, fontweight='bold')
    
    # Agregar valores en celdas
    for i in range(3):
        for j in range(len(numeric_drift_top)):
            text = ax3.text(j, i, f'{metrics_matrix.iloc[i, j]:.3f}',
                          ha="center", va="center", color="black", fontsize=8, fontweight='bold')
    
    # Colorbar
    cbar = plt.colorbar(im, ax=ax3)
    cbar.set_label('Magnitud de Drift', rotation=270, labelpad=20, fontweight='bold')

# 4. Timeline simulado de drift (simulación con ruido)
ax4 = fig.add_subplot(gs[2, :])
time_points = pd.date_range(end=datetime.now(), periods=10, freq='W')
# Simular evolución temporal del drift promedio
psi_mean = drift_df[drift_df['Tipo'] == 'Numérica']['PSI'].mean()
drift_timeline = [psi_mean * (0.8 + 0.4 * np.random.random()) for _ in range(10)]

ax4.plot(time_points, drift_timeline, marker='o', linewidth=2, markersize=8, color='blue', label='PSI Promedio')
ax4.axhline(y=0.1, color='orange', linestyle='--', linewidth=2, label='Umbral Moderado', alpha=0.7)
ax4.axhline(y=0.2, color='red', linestyle='--', linewidth=2, label='Umbral Crítico', alpha=0.7)
ax4.fill_between(time_points, 0, 0.1, color='green', alpha=0.1)
ax4.fill_between(time_points, 0.1, 0.2, color='orange', alpha=0.1)
ax4.fill_between(time_points, 0.2, max(drift_timeline + [0.25]), color='red', alpha=0.1)

ax4.set_xlabel('Período de Tiempo', fontweight='bold')
ax4.set_ylabel('PSI Promedio', fontweight='bold')
ax4.set_title('Evolución Temporal del Drift (Simulación)', fontsize=12, fontweight='bold')
ax4.legend(loc='upper left')
ax4.grid(alpha=0.3)
ax4.tick_params(axis='x', rotation=45)

plt.suptitle('🎯 DASHBOARD DE MONITOREO DE DATA DRIFT', fontsize=16, fontweight='bold', y=0.995)
plt.show()

print("✅ Dashboard de monitoreo generado")

## 8. Recomendaciones y Acciones

Generar recomendaciones automáticas basadas en el análisis de drift.

In [6]:
print("\n" + "="*80)
print("💡 RECOMENDACIONES Y PLAN DE ACCIÓN")
print("="*80 + "\n")

# Calcular métricas agregadas
total_vars = len(drift_df)
pct_critical = (len(critical_vars) / total_vars) * 100
pct_moderate = (len(moderate_vars) / total_vars) * 100
pct_ok = (len(ok_vars) / total_vars) * 100

# Sistema de scoring de riesgo
risk_score = (len(critical_vars) * 3 + len(moderate_vars) * 1)
max_risk = total_vars * 3
risk_percentage = (risk_score / max_risk) * 100

print(f"📊 EVALUACIÓN DE RIESGO GENERAL:")
print(f"   Score de Riesgo: {risk_score}/{max_risk} ({risk_percentage:.1f}%)")
print(f"   • {pct_critical:.1f}% variables críticas")
print(f"   • {pct_moderate:.1f}% variables moderadas")
print(f"   • {pct_ok:.1f}% variables estables\n")

# Determinar nivel de riesgo general
if risk_percentage < 10:
    risk_level = "🟢 BAJO"
    action = "Continuar monitoreo regular"
elif risk_percentage < 30:
    risk_level = "🟡 MEDIO"
    action = "Aumentar frecuencia de monitoreo"
else:
    risk_level = "🔴 ALTO"
    action = "Reentrenamiento del modelo URGENTE"

print(f"🎯 NIVEL DE RIESGO: {risk_level}")
print(f"📋 ACCIÓN RECOMENDADA: {action}\n")

# Recomendaciones específicas
print("="*80)
print("RECOMENDACIONES ESPECÍFICAS:")
print("="*80 + "\n")

if len(critical_vars) > 0:
    print("🚨 ACCIÓN INMEDIATA (Variables Críticas):")
    print("   1. Investigar causas del drift en las siguientes variables:")
    for _, row in critical_vars.head(5).iterrows():
        print(f"      • {row['Variable']}")
    print("\n   2. Considerar reentrenamiento del modelo con datos actualizados")
    print("   3. Revisar pipeline de preprocesamiento")
    print("   4. Validar calidad de datos actuales\n")

if len(moderate_vars) > 0:
    print("⚠️  MONITOREO CONTINUO (Variables Moderadas):")
    print("   1. Aumentar frecuencia de muestreo para estas variables:")
    for _, row in moderate_vars.head(5).iterrows():
        print(f"      • {row['Variable']}")
    print("\n   2. Establecer alertas tempranas")
    print("   3. Documentar cambios observados\n")

if len(ok_vars) == total_vars:
    print("✅ ESTADO ÓPTIMO:")
    print("   • Todas las variables están dentro de umbrales aceptables")
    print("   • Mantener frecuencia de monitoreo actual")
    print("   • No se requieren acciones correctivas\n")

# Recomendaciones de periodicidad
print("="*80)
print("📅 PERIODICIDAD DE MONITOREO RECOMENDADA:")
print("="*80)
if risk_percentage >= 30:
    print("   🔴 Monitoreo: DIARIO")
    print("   🔴 Revisión: Cada 3 días")
elif risk_percentage >= 10:
    print("   🟡 Monitoreo: SEMANAL")
    print("   🟡 Revisión: Cada 2 semanas")
else:
    print("   🟢 Monitoreo: QUINCENAL")
    print("   🟢 Revisión: Mensual")

print("\n" + "="*80)
print("✅ Análisis de drift completado exitosamente")
print("="*80)


💡 RECOMENDACIONES Y PLAN DE ACCIÓN

📊 EVALUACIÓN DE RIESGO GENERAL:
   Score de Riesgo: 0/99 (0.0%)
   • 0.0% variables críticas
   • 0.0% variables moderadas
   • 100.0% variables estables

🎯 NIVEL DE RIESGO: 🟢 BAJO
📋 ACCIÓN RECOMENDADA: Continuar monitoreo regular

RECOMENDACIONES ESPECÍFICAS:

✅ ESTADO ÓPTIMO:
   • Todas las variables están dentro de umbrales aceptables
   • Mantener frecuencia de monitoreo actual
   • No se requieren acciones correctivas

📅 PERIODICIDAD DE MONITOREO RECOMENDADA:
   🟢 Monitoreo: QUINCENAL
   🟢 Revisión: Mensual

✅ Análisis de drift completado exitosamente
